In [4]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/home/ec2-user/.kaggle’: File exists


In [5]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 KB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 KB 5.6 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=39e6b953c00d42a6f56e97ebf2a690860a4fade2331d88827371f1a13c8adba9
  Stored in directory: /home/ec2-user/.cache/pip/wheels/29/da/11/144cc25aebdaeb4931b231e25fd34b394e6a5725cbb2f50106
Successfully built kaggle
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [6]:
! cp kaggle.json ~/.kaggle/

In [10]:
!kaggle datasets download -d gaveshjain/ford-sentence-classifiaction-dataset

ford-sentence-classifiaction-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [12]:
!unzip ford-sentence-classifiaction-dataset

Archive:  ford-sentence-classifiaction-dataset.zip
  inflating: sample_submission.csv   
  inflating: test_data.csv           
  inflating: train_data.csv          


In [2]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer

In [3]:
df = pd.read_csv('ford-sentence-classifiaction-dataset/train_data.csv')

In [4]:
df_test = pd.read_csv('ford-sentence-classifiaction-dataset/test_data.csv')

In [5]:
df_sample = pd.read_csv('ford-sentence-classifiaction-dataset/sample_submission.csv')

In [32]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59002 entries, 0 to 60114
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    59002 non-null  int64 
 1   Sentence_id   59002 non-null  object
 2   New_Sentence  59002 non-null  object
 3   Type          59002 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [36]:
df.head()

,Unnamed: 0,Sentence_id,New_Sentence,Type
0,0,GERRES15609,author and or review architecture design and o...,Responsibility
1,1,PHERES15784,should be able to develop custom dynamic shape...,Responsibility
2,2,GERREQ10457,experience in working crosslly with a larger e...,Requirement
3,3,GERSKL27235,previous business experience including but not...,Skill
4,4,HONSSK18415,delivering fast and right the first time,SoftSkill


In [37]:
print(df.head().to_string())

   Unnamed: 0  Sentence_id                                                                                                                                                                 New_Sentence            Type
0           0  GERRES15609  author and or review architecture design and other technical documents ensuring high quality deliverables and systems development across tech stacks and applications teams  Responsibility
1           1  PHERES15784                                                                        should be able to develop custom dynamic shape object script and validation and testing with database  Responsibility
2           2  GERREQ10457                                                                      experience in working crosslly with a larger engineering organization and multiple sites highly desired     Requirement
3           3  GERSKL27235                                previous business experience including but not limited to business management 

In [38]:
print(df_test.head().to_string())

   Unnamed: 0  Sentence_id                                                                                                                                                                                                                           New_Sentence  predict_prob          target            Type
0           0  UAERES16346             Collaborate across all of DB&T practices and offerings in AI, Data, IoT  and Cloud platforms to ensure we grow share in world class digital innovation, mobile/web applications helping clients reinvent their businesses.      0.006665  Responsibility  Responsibility
1           1  COGREQ15586                                                                                                                                                                        Strong  technology expertise in Identity and Access Management.      0.176393     Requirement     Requirement
2           2  UAEREQ12722                                                              

In [17]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15029 entries, 0 to 15028
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    15029 non-null  int64 
 1   Sentence_id   15029 non-null  object
 2   New_Sentence  14748 non-null  object
dtypes: int64(1), object(2)
memory usage: 352.4+ KB


In [21]:
df_test['New_Sentence'].head()

0    Collaborate across all of DB&T practices and o...
1    Strong  technology expertise in Identity and A...
2         Strong knowledge on Service  Virtualization.
3    Architect scalable data processing and analyti...
4    Map client organization, build outstanding rel...
Name: New_Sentence, dtype: object

In [22]:
df_test.dropna(inplace=True)

In [7]:
import re
#data preprocessing by regualar expression 
def dataPreprocessing(text):
    text=re.sub(r'[^\w\s\t\']',' ',text)
    text=re.sub(r' +',' ',text)
    text = text.replace(r'/',' ')
    return text.strip().lower()

In [8]:
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
x=[]
def datacleaning(New_Sentence):
    remove_punctuation=[char for char in New_Sentence if char not in string.punctuation]

    remove_punctuation=''.join(remove_punctuation)
    
    return [word for word in remove_punctuation.split() if word.lower() not in stopwords.words('english')]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
df['New_Sentence']=df['New_Sentence'].map(dataPreprocessing)
print(df.iloc[:,2].apply(datacleaning))

0        [author, review, architecture, design, technic...
1        [able, develop, custom, dynamic, shape, object...
2        [experience, working, crosslly, larger, engine...
3        [previous, business, experience, including, li...
4                   [delivering, fast, right, first, time]
                               ...                        
60110    [position, utilize, program, management, skill...
60111    [addition, individual, responsible, managing, ...
60112                     [good, problem, solving, skills]
60113                             [good, excel, knowledge]
60114    [bachelors, degree, electrical, engineering, m...
Name: New_Sentence, Length: 59002, dtype: object


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
transformer=CountVectorizer(analyzer=datacleaning).fit(df['New_Sentence'])
#transformer.vocabulary_

In [12]:
map_link=transformer.transform(df['New_Sentence'])
#print(map_link)

In [40]:
#most significant words
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer().fit(map_link)
#print(tfidf_transformer)
title_tfidf=tfidf_transformer.transform(map_link)
#print(title_tfidf)
print(title_tfidf.shape)

(59002, 21244)


In [41]:
df_test['New_Sentence']=df_test['New_Sentence'].map(dataPreprocessing)
print(df.iloc[:,2].apply(datacleaning))

0        [author, review, architecture, design, technic...
1        [able, develop, custom, dynamic, shape, object...
2        [experience, working, crosslly, larger, engine...
3        [previous, business, experience, including, li...
4                   [delivering, fast, right, first, time]
                               ...                        
60110    [position, utilize, program, management, skill...
60111    [addition, individual, responsible, managing, ...
60112                     [good, problem, solving, skills]
60113                             [good, excel, knowledge]
60114    [bachelors, degree, electrical, engineering, m...
Name: New_Sentence, Length: 59002, dtype: object


In [43]:
map_link_test=transformer.transform(df_test['New_Sentence'])
#most significant words
from sklearn.feature_extraction.text import TfidfTransformer
tfidf__test_transformer=TfidfTransformer().fit(map_link_test)
#print(tfidf_transformer)
title_tfidf_test=tfidf_transformer.transform(map_link_test)
#print(title_tfidf)
print(title_tfidf_test.shape)

(14748, 21244)


In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model, naive_bayes, svm
#model=svm.LinearSVC(C=1.0).fit(title_tfidf,df['New_Sentence'])
model=MultinomialNB().fit(title_tfidf,df['Type'])

In [39]:
all_predictions=model.predict(title_tfidf)
print(all_predictions)

['Responsibility' 'Responsibility' 'Requirement' ... 'SoftSkill' 'Skill'
 'Education']


In [44]:
y_predict = model.predict(title_tfidf_test)      
y_prob = model.predict_proba(title_tfidf_test)[:,1]
df_test['predict_prob']= y_prob
df_test['Type']= y_predict
final=df_test[['New_Sentence','Type']].reset_index(drop=True)
print(final.head())

                                        New_Sentence            Type
0  collaborate across all of db t practices and o...  Responsibility
1  strong technology expertise in identity and ac...     Requirement
2         strong knowledge on service virtualization           Skill
3  architect scalable data processing and analyti...  Responsibility
4  map client organization build outstanding rela...  Responsibility


In [46]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
final=df[['New_Sentence','Type']].reset_index(drop=True)
X_train, X_test, y_train, y_test = train_test_split(df['New_Sentence'],df['Type'],test_size=0.2,shuffle=True)

In [54]:
X_test.head()
map_link_X_test=transformer.transform(X_test)
#most significant words
from sklearn.feature_extraction.text import TfidfTransformer
tfidf__test_transformer=TfidfTransformer().fit(map_link_X_test)
#print(tfidf_transformer)
title_tfidf_X_test=tfidf_transformer.transform(map_link_X_test)
#print(title_tfidf)
print(title_tfidf_X_test.shape)

(11801, 21244)


In [58]:
y_predict = model.predict(title_tfidf_X_test)
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

                precision    recall  f1-score   support

     Education       0.81      0.74      0.78       875
    Experience       0.84      0.93      0.88      1855
   Requirement       0.61      0.63      0.62      2664
Responsibility       0.77      0.92      0.84      3092
         Skill       0.74      0.39      0.51      1343
     SoftSkill       0.77      0.70      0.74      1972

      accuracy                           0.75     11801
     macro avg       0.76      0.72      0.73     11801
  weighted avg       0.75      0.75      0.74     11801

Confusion Matrix: [[ 650   36  124   48   14    3]
 [  13 1717   63   40   21    1]
 [ 130  126 1683  319  100  306]
 [   1   21  145 2853   24   48]
 [   6  128  420  214  527   48]
 [   0   14  318  232   22 1386]]


In [64]:
submission = pd.DataFrame({"Sentence_id": df_test["Sentence_id"],"New_Sentence": df_test["New_Sentence"], "Type": df_test["Type"]})
submission.to_csv('submission.csv', index=False)

In [66]:
submission = pd.read_csv('submission.csv')
submission.head(10)

,Sentence_id,New_Sentence,Type
0,UAERES16346,collaborate across all of db t practices and o...,Responsibility
1,COGREQ15586,strong technology expertise in identity and ac...,Requirement
2,UAEREQ12722,strong knowledge on service virtualization,Skill
3,COGSKL29155,architect scalable data processing and analyti...,Responsibility
4,PHERES12551,map client organization build outstanding rela...,Responsibility
5,INDRES5908,assure compliance and recommend actions to imp...,Responsibility
6,UAESKL39669,we valueknowledge of purchasing system oracle ...,Requirement
7,HONSSK10706,lead a global finance team in driving financia...,Responsibility
8,COGRES1719,strong physical architecture concepts infrastr...,Skill
9,HONREQ3632,ability to handle complex and sensitive custom...,Requirement


In [67]:
print("MultinomialNB Accuracy Score -> ",accuracy_score(y_predict, y_test)*100)

MultinomialNB Accuracy Score ->  74.70553342937039
